# 2022-49 Mode Choice and SB743 VMT QC

In [2]:
import pandas as pd
import sqlalchemy as sql

ddam = sql.create_engine('mssql+pymssql://DDAMWSQL16/demographic_warehouse')

## Mode Choice Report

### Study Area Trips section

In [3]:
# The list of MGRAs that define the downtown area
mgra = pd.read_csv("T:/ABM/SB/Enhancement_ABM2+/data/PowerBI/ModeChoiceReport/service_bureau_mode_choice_reports/QA/cicpa_downtown_mgra.csv").values
mgra = tuple([str(y) for z in mgra for y in z])

# The scenario_id to use
SCENARIO_ID = 100

# geography_set_id = 4 corresponds to "zone split for Seaport Village land use converter test 
# 6.1.2022". This value is used because in [abm_14_3_0].[dimension].[scenario], scenario_id=100 
# corresponds to geography_set_id = 4
GEOGRAPHY_SET_ID = 4

In [11]:
# Double check that the MGRAs which define the downtown area are correct
downtown_mgras = pd.read_sql(f"""
    SELECT mgra_13
    FROM [abm_14_3_0].[dimension].[geography]
    WHERE cicpa_2016_name='Downtown'
        AND mgra_13 != 'Not Applicable'
        AND geography_set_id = 4
""", con=ddam.connect())
mgra == tuple(downtown_mgras["mgra_13"])

True

In [3]:
# Get all the person trips from SQL filtered by scenario_id, the list of MGRAs in the jurisdiction,
# and the correct geography_set_id
person_trips = pd.read_sql(f"""
    SELECT 
        geo_o.mgra_13 AS origin_mgra, 
        geo_d.mgra_13 AS dest_mgra, 
        trip.*
    FROM [abm_14_3_0].[fact].[person_trip] AS trip
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_o ON
        trip.geography_trip_origin_id = geo_o.geography_id
    LEFT JOIN [abm_14_3_0].[dimension].[geography] AS geo_d ON
        trip.geography_trip_destination_id = geo_d.geography_id
    WHERE scenario_id={SCENARIO_ID}
        AND (geo_o.mgra_13 IN {mgra} OR geo_d.mgra_13 IN {mgra})
        AND geo_o.geography_set_id = {GEOGRAPHY_SET_ID}
        AND geo_d.geography_set_id = {GEOGRAPHY_SET_ID}
    """, con=ddam.connect())

In [4]:
# Intra-Zonal = trip starts and ends in the same MGRA
intra_zonal = person_trips[person_trips["origin_mgra"] == person_trips["dest_mgra"]].copy(deep=True)

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Intra-Zonal values")
print(f"{'Pre filter trips:' : <20}", intra_zonal.shape[0])
intra_zonal = intra_zonal[intra_zonal["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", intra_zonal.shape[0])

Intra-Zonal values
Pre filter trips:    20170
Post filter trips:   18392


In [5]:
# Inter-Zonal = trip starts and ends in the same jurisdiction, but not the same MGRA
inter_zonal = person_trips[person_trips["origin_mgra"] != person_trips["dest_mgra"]].copy(deep=True)
inter_zonal = inter_zonal[inter_zonal["origin_mgra"].isin(mgra) & inter_zonal["dest_mgra"].isin(mgra)]

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Inter-Zonal values")
print(f"{'Pre filter trips:' : <20}", inter_zonal.shape[0])
inter_zonal = inter_zonal[inter_zonal["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", inter_zonal.shape[0])

Inter-Zonal values
Pre filter trips:    342230
Post filter trips:   307715


In [6]:
# Inter-Study Area = trip starts/ends in the jurisdiction, but ends/starts outside the jurisdiction
inter_study = person_trips.copy(deep=True)
inter_study = inter_study[
    (inter_study["origin_mgra"].isin(mgra) & ~inter_study["dest_mgra"].isin(mgra)) |
    (~inter_study["origin_mgra"].isin(mgra) & inter_study["dest_mgra"].isin(mgra))]

# For some reason, some filtering is done on model_trip_id for no (to me at least) discernable 
# reason
print("Inter-Study Area values")
print(f"{'Pre filter trips:' : <20}", inter_study.shape[0])
inter_study = inter_study[inter_study["model_trip_id"].isin([1, 2, 4])]
print(f"{'Post filter trips:' : <20}", inter_study.shape[0])

Inter-Study Area values
Pre filter trips:    700524
Post filter trips:   616203


## VMT

### VMT Per Employee by Geography